In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from textblob import TextBlob

In [2]:
def open_pickle(path):
    with open(path, 'rb') as f:
        X = pickle.load(f)
    return X

In [4]:
#Load dataset of [whole corpus]

X_train_original = open_pickle('../data/imdb/imdb_original_preprocessed_xtrain.pickle')
X_test_original = open_pickle('../data/imdb/imdb_original_preprocessed_xtest.pickle')
y_train_original = open_pickle('../data/imdb/imdb_original_preprocessed_ytrain.pickle')
y_test_original = open_pickle('../data/imdb/imdb_original_preprocessed_ytest.pickle')

#Load dataset of sentence [+/-]

X_train_np_sentence = open_pickle('../data/imdb-sentence/imdb_sentence_np_xtrain.pickle')
X_test_np_sentence = open_pickle('../data/imdb-sentence/imdb_sentence_np_xtest.pickle')
y_train_np_sentence = open_pickle('../data/imdb-sentence/imdb_sentence_np_ytrain.pickle')
y_test_np_sentence = open_pickle('../data/imdb-sentence/imdb_sentence_np_ytest.pickle')

In [26]:
# Count Vectorizer on rel,unrel dataset
# Question : Why rel/unrel? Because it trained as the first step? 
# Any advantages on more vocabulary?

token = r"(?u)\b[\w\'/]+\b"
tf_vectorizer = CountVectorizer(lowercase=True, max_df=1.0, min_df=5, binary=True, token_pattern=token)
tf_vectorizer.set_params(ngram_range=(1,1))

# whole imdb corpus
X_train_original_bow = tf_vectorizer.fit_transform(X_train_original)
X_test_original_bow = tf_vectorizer.transform(X_test_original)

# positive/negative sentence
X_train_np_bow = tf_vectorizer.transform(X_train_np_sentence)
X_test_np_bow = tf_vectorizer.transform(X_test_np_sentence) 

words = tf_vectorizer.get_feature_names()
words = np.array(words)

In [27]:
# Initial variable
random_state = 42
C = 1

# using whole corpus
clf_i = LogisticRegression(random_state=random_state, C=C)
clf_i.fit(X_train_original_bow, y_train_original)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Approach

In [102]:
idx = 4

document = np.vstack([X_train_original[idx], X_train_original[idx]])
sentences = TextBlob(X_train_original[idx]).raw_sentences
# " ".join(sentences)

In [47]:
document = document.flatten()
len(document)

2

In [28]:
len(sentences)

27

In [48]:
x_doc = tf_vectorizer.transform(document)
x = tf_vectorizer.transform(sentences)

In [49]:
x.shape

(27, 26266)

In [50]:
x_doc.shape

(2, 26266)

In [96]:
x = x.todense()

In [97]:
x.shape

(26, 26266)

In [98]:
sent = []

prob_ = clf_i.predict_proba(x_doc)
prob = prob_[0]

for i in range(x.shape[0]):
    sentences_temp = np.delete(sentences, i, 0)
    sent.append(" ".join(sentences_temp))

In [117]:

if prob[0] > prob[1]:
    base = 0
else:
    base = 1

In [99]:
len(sent)

26

In [104]:
prob_drop = []

x_drop = tf_vectorizer.transform(sent)
x_drop.shape

(26, 26266)

In [105]:
probs = clf_i.predict_proba(x_drop)

In [111]:
prob_drop = np.absolute(probs - prob)

In [118]:
drops = np.absolute(probs[:,base] - prob[base])

In [120]:
np.argmax(drops)

12

In [121]:
sentences[12]

'he only occasionally gives outburst after quietly putting up with more than most could stand under such circumstance.'

Gotcha!